In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

from azureml.pipeline.steps import PythonScriptStep

from azureml.pipeline.core.graph import PipelineParameter

from azureml.pipeline.core import Pipeline, PipelineEndpoint
from azureml.core import Experiment

import datetime as dt


In [2]:
ws = Workspace.from_config()

compute_name = "logistic-compute"
vm_size = "Standard_DS12_v2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else: 
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                idle_seconds_before_scaledown=3600*2,
                                                                min_nodes=0,
                                                                max_nodes=4)
    # create the compute target
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

Found compute target: logistic-compute


In [3]:
aml_run_config = RunConfiguration()
# `compute_target` as defined in "Azure Machine Learning compute" section above
aml_run_config.target = compute_target
    

aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        pip_packages=['pandas','scikit-learn', 'sqlalchemy', 'openpyxl', 'pulp'],
        conda_packages=['pyodbc'],
        pin_sdk_version=False
    )

In [4]:
# Creating pipeline in userfriendly in-face

entry_point = "Model_v0.1.py"

year = PipelineParameter(
  name="year",
  default_value=0,)

month =  PipelineParameter(
    name="month",
    default_value=0)

version = PipelineParameter(
    name="version",
    default_value=0)

startContractLsd = PipelineParameter(
    name="startContractLsd",
    default_value='2020-10-01')

endContractLsd = PipelineParameter(
    name="endContractLsd",
    default_value='2020-12-31')

planningDate = PipelineParameter(
    name="planningDate",
    default_value='2020-10-01')

runNumber = PipelineParameter(
    name="runNumber",
    default_value=0)

user = PipelineParameter(
  name="user",
  default_value='Andriy')


step = PythonScriptStep(
    script_name=entry_point,
    arguments=[user, year, month, version, startContractLsd, endContractLsd,
                      planningDate, runNumber],
    compute_target=compute_target,
    runconfig=aml_run_config,
    allow_reuse=False
)

pipeline1 = Pipeline(workspace=ws, steps=[[step]])

In [5]:
published_pipeline = pipeline1.publish(name='optimization_version_1.33', description='Run optization model', version='1.33')

Created step Model_v0.1.py [8ec3a91d][e3323eb6-671d-461f-931e-4830b2fd75f5], (This step will run and generate new outputs)


In [6]:
# # run only ones
# published_endpoint = PipelineEndpoint.publish(workspace=ws,
#      name="DP4_versioned",
#      pipeline=pipeline1,                                       
#      description="DP4 Azovstal model versioned pipeline")
# # published_pipeline

In [7]:
published_endpoint1 = PipelineEndpoint.get(workspace=ws, name='Optimization')
published_endpoint1.add_default(published_pipeline)